# FPUS23 — RF-DETR on Colab

Train RF-DETR on FPUS23 COCO. Optional: build a Balanced RF-DETR view for fair train rebalancing.

In [ ]:
# Install RF-DETR
!pip -q install rfdetr pycocotools


In [ ]:
# Save to Drive (optional)
SAVE_TO_DRIVE = True
DRIVE_DIR = '/content/drive/MyDrive/FPUS23_transformers'
if SAVE_TO_DRIVE:
    from google.colab import drive; drive.mount('/content/drive', force_remount=True)
    import pathlib as p; p.Path(DRIVE_DIR).mkdir(parents=True, exist_ok=True)
    print('Saving to:', DRIVE_DIR)
else:
    print('Saving to local /content/outputs')


In [ ]:
# Toggle: build a Balanced RF-DETR dataset view
USE_BALANCED = True
print('USE_BALANCED=', USE_BALANCED)
COCO_ROOT = '/content/fpus23_project/dataset/fpus23_coco'
OUT_ROOT  = '/content/fpus23_project/dataset/fpus23_coco_rfdetr'
BAL_ROOT  = '/content/fpus23_project/dataset/fpus23_coco_rfdetr_balanced'


In [ ]:
# Build the standard RF-DETR view (train/valid/test with _annotations.coco.json)
!python 'New folder/scripts/train_rfdetr_fpus23.py' --project-root /content/fpus23_project --epochs 1 --batch 1 --grad-accum 1 --lr 1e-4 --ema 0
# We only ran 1 epoch to create the expected folder structure; real training below.


In [ ]:
# (Optional) Build a Balanced RF-DETR view
import os, shutil, pathlib as p, json
if USE_BALANCED:
    src_bal_imgs = p.Path(COCO_ROOT)/'images_balanced'/'train'
    src_bal_json = p.Path(COCO_ROOT)/'annotations'/'train_balanced.json'
    dst = p.Path(BAL_ROOT); (dst/'train').mkdir(parents=True, exist_ok=True)
    # Copy balanced images into the RF-DETR 'train'
    for im in src_bal_imgs.glob('*.png'):
        target = (dst/'train'/im.name)
        if not target.exists(): shutil.copy(im, target)
    # Copy val/test from standard view
    for split in ['valid','test']:
        base = p.Path(OUT_ROOT)/split
        (dst/split).mkdir(parents=True, exist_ok=True)
        for im in base.glob('*.png'):
            t = (dst/split/im.name)
            if not t.exists(): shutil.copy(im, t)
    # Write RF-DETR JSON names
    shutil.copy(src_bal_json, (dst/'train'/'_annotations.coco.json'))
    shutil.copy(p.Path(COCO_ROOT)/'val.json', (dst/'valid'/'_annotations.coco.json'))
    shutil.copy(p.Path(COCO_ROOT)/'test.json', (dst/'test'/'_annotations.coco.json'))
    print('Balanced RF-DETR view built at:', BAL_ROOT)


## Train RF-DETR
Use the python API directly so we can pass the balanced dataset_dir when requested.

In [ ]:
from rfdetr import RFDETRBase
import pathlib as p
dataset_dir = p.Path(BAL_ROOT if USE_BALANCED else OUT_ROOT)
output_dir  = p.Path(DRIVE_DIR)/('rfdetr_fpus23_balanced' if USE_BALANCED else 'rfdetr_fpus23')
output_dir.mkdir(parents=True, exist_ok=True)
model = RFDETRBase()
model.train(dataset_dir=str(dataset_dir), epochs=20, batch_size=4, grad_accum_steps=4, lr=1e-4, output_dir=str(output_dir), weight_decay=1e-4, use_ema=True)
print('Saved to', output_dir)
